# 8. Model Deployment Using FastAPI

In [ ]:
# Install necessary packages
!pip install fastapi uvicorn pyngrok nest-asyncio streamlit

In [ ]:
# Standard library imports
import os
import pickle
import requests
from threading import Thread

# Third-party imports
import pandas as pd
import uvicorn
import nest_asyncio
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok

## Deploying a Prediction Service with FastAPI and Ngrok

This section sets up a RESTful API using **FastAPI** to serve predictions from the trained machine learning model. The API is exposed to the public using **Ngrok** for remote accessibility.

### Key Components:

1. **Model Loading**:
   - The trained model is loaded from a pickle file (`best_model.pkl`) stored in the Colab environment (since the notebooks are different , we would need to manually select the best model and utilize it here)
   - Ensure that the file exists in the specified path before running the code. You can download a sample best model from here https://github.com/Guardian99/ISB-CT1_GR03/blob/main/Models/best_model.pkl

2. **FastAPI Initialization**:
   - The `FastAPI` application is instantiated to define and serve the API endpoints.

3. **Input Schema**:
   - The `PredictionInput` class (a **Pydantic** model) defines the expected input structure for prediction requests. This ensures type validation and proper request formatting.

4. **Prediction Endpoint**:
   - A POST endpoint (`/predict`) is created to handle prediction requests.
   - The input data is converted to a Pandas DataFrame before being passed to the model's `predict` method.
   - The API returns the predicted class as a JSON response.

5. **Run FastAPI in Colab**:
   - `nest_asyncio` is used to allow the FastAPI server to run alongside the Colab notebook environment.
   - The server runs locally on port `8000`.

6. **Ngrok Integration**:
   - Ngrok is used to expose the local FastAPI server to a public URL.
   - The Ngrok authentication token is required to secure the tunnel.
   - The generated public URL is printed for user access.

7. **Save Public URL**:
   - The public URL is saved to a file (`fastapi_url.txt`) for easy retrieval or use in other services like Streamlit.

### Example Outputs:
- **FastAPI Public URL**: https://57b4-34-55-151-170.ngrok-free.app


### Benefits:
- **Remote Prediction Service**:
- Enables real-time predictions from the trained model, accessible from any device with the public URL.
- **Validation**:
- The use of Pydantic models ensures that incoming data adheres to the expected schema, reducing potential errors.
- **Scalable Deployment**:
- Can be extended or integrated with other applications for real-world use cases.

### Applications:
This setup is ideal for deploying machine learning models for:
- Employee attrition prediction.
- Providing predictions as a service for other applications.
- Quick and collaborative model testing and validation.


In [ ]:
# Load the model
MODEL_PATH = "/content/best_model.pkl"  # Ensure this file exists in the Colab environment
with open(MODEL_PATH, "rb") as f:
    model = pickle.load(f)

# Initialize FastAPI app
app = FastAPI()

# Define the input schema
class PredictionInput(BaseModel):
    Education: str
    JoiningYear: int
    City: str
    PaymentTier: int
    Age: int
    Gender: str
    EverBenched: int
    ExperienceInCurrentDomain: int

@app.post("/predict")
def predict(input_data: PredictionInput):
    """Handle prediction requests."""
    input_df = pd.DataFrame([input_data.dict()])
    prediction = model.predict(input_df)
    return {"prediction": int(prediction[0])}

# Allow FastAPI to run in Colab
nest_asyncio.apply()

# Start FastAPI server on port 8000
def run_fastapi():
    """Run the FastAPI server."""
    uvicorn.run(app, host="0.0.0.0", port=8000)

fastapi_thread = Thread(target=run_fastapi, daemon=True)
fastapi_thread.start()

# Expose FastAPI through ngrok
auth_token = '2qAc8RDV3pPFCephUHKYQgjLxZE_3vFmXHsh66CVDFkXzAmG9'
ngrok.set_auth_token(auth_token)
ngrok_tunnel = ngrok.connect(8000)
fastapi_url = ngrok_tunnel.public_url
print(f"FastAPI public URL: {fastapi_url}")

# Save FastAPI URL to a file for Streamlit to use
with open("fastapi_url.txt", "w") as f:
    f.write(fastapi_url)

INFO:     Started server process [207]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


FastAPI public URL: https://ad89-34-138-219-47.ngrok-free.app


## Testing the FastAPI Prediction Endpoint

This section demonstrates how to test the deployed FastAPI service by sending a prediction request and reviewing the server's response.

### Key Steps:

1. **Retrieve FastAPI Public URL**:
   - The public URL for the FastAPI server is loaded from the `fastapi_url.txt` file, ensuring the correct endpoint is used for the request.

2. **Prepare the Payload**:
   - A JSON payload is defined with input data conforming to the `PredictionInput` schema. This data includes both categorical and numerical features required by the model.

3. **Send a POST Request**:
   - A POST request is made to the `/predict` endpoint using the `requests` library.
   - The JSON payload is sent as the request body.

4. **Response Handling**:
   - The response from the FastAPI server is parsed and printed:
     - **Status Code**: Indicates whether the request was successful (e.g., `200 OK`).
     - **Headers**: Provides metadata about the response.
     - **Response Text**: Displays the actual prediction returned by the server.

### Example Output:
- **Status Code**: `200` (indicating success)
- **Headers**: Metadata, such as content type.
- **Response Text**: JSON data, e.g., `{"prediction": 1}`, where `1` indicates a positive prediction for employee attrition.

### Benefits:
- **End-to-End Testing**:
  - Verifies that the FastAPI service processes requests correctly and returns predictions as expected.
- **Real-World Simulation**:
  - Mimics client-side behavior to ensure the API works seamlessly for external users or applications.

### Common Use Cases:
- **Debugging**:
  - Identifies issues in the prediction pipeline, such as mismatched input schemas or model errors.
- **Integration Testing**:
  - Ensures the API integrates well with client-side applications or other services.
- **Performance Monitoring**:
  - Evaluates response times and reliability under typical usage conditions.




In [ ]:
# Testing the fastapi endpoint
# Load FastAPI URL from file
with open("fastapi_url.txt", "r") as file:
    url = file.read().strip()

# Define the payload for the prediction request
payload = {
    "Education": "Bachelors",
    "JoiningYear": 2015,
    "City": "Mumbai",
    "PaymentTier": 2,
    "Age": 30,
    "Gender": "Male",
    "EverBenched": 0,  # Representing "No" as 0 for alignment with the schema
    "ExperienceInCurrentDomain": 5
}

# Make a POST request to the FastAPI server
response = requests.post(url + "/predict", json=payload)

# Print the status code
print(f"Status Code: {response.status_code}")

# Print the headers
print(f"Headers: {response.headers}")

# Print the content of the response (the actual prediction)
print(f"Response Text: {response.text}")

<ipython-input-3-1d6334c4a201>:23: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  input_df = pd.DataFrame([input_data.dict()])
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-22' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 162, in startup
    server = await loop.create_server(
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1519, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most r

INFO:     34.138.219.47:0 - "POST /predict HTTP/1.1" 200 OK
Status Code: 200
Headers: {'Content-Length': '16', 'Content-Type': 'application/json', 'Date': 'Fri, 13 Dec 2024 17:33:31 GMT', 'Ngrok-Agent-Ips': '34.138.219.47', 'Server': 'uvicorn'}
Response Text: {"prediction":0}
